In [4]:
import requests
import pandas as pd

def get_imf_gdp_table():
    # 1. IMF DataMapper API 엔드포인트 (NGDPD: 명목 GDP, US Dollars)
    # 이 API는 최신 WEO(World Economic Outlook) 데이터를 반환합니다.
    url = "https://www.imf.org/external/datamapper/api/v1/NGDPD"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # 2. 데이터 구조 파싱
        # values 구조: data['values']['NGDPD'][국가코드][연도]
        gdp_data = data.get('values', {}).get('NGDPD', {})
        countries_meta = data.get('countries', {})

        # 3. 최신 연도 확인 (가장 마지막에 업데이트된 연도 추출)
        # 데이터 구조상 국가별로 연도가 다를 수 있으므로 전체에서 가장 최신 연도를 찾습니다.
        all_years = []
        for c_code in gdp_data:
            all_years.extend(gdp_data[c_code].keys())
        latest_year = max(all_years)

        print(f"--- {latest_year}년 기준 IMF GDP 데이터를 추출합니다 ---")

        # 4. 테이블 구성을 위한 리스트 생성
        table_list = []
        for country_code, years in gdp_data.items():
            if latest_year in years:
                country_name = countries_meta.get(country_code, {}).get('label', country_code)
                gdp_val = years[latest_year]

                table_list.append({
                    'Country': country_name,
                    'GDP (1B USD)': gdp_val
                })

        # 5. 데이터프레임 생성 및 가공
        df = pd.DataFrame(table_list)

        # GDP 기준 내림차순 정렬
        df = df.sort_values(by='GDP (1B USD)', ascending=False).reset_index(drop=True)

        # 소수점 2자리 포맷팅 (단위는 API 기본값이 Billions이므로 그대로 유지)
        df['GDP (1B USD)'] = df['GDP (1B USD)'].round(2)

        return df

    except Exception as e:
        print(f"데이터를 가져오는 중 오류가 발생했습니다: {e}")
        return None

# 실행 및 결과 출력
gdp_table = get_imf_gdp_table()

if gdp_table is not None:
    # 상위 20개 국가 확인
    print(gdp_table.head(20))

    # CSV 또는 JSON으로 저장 가능 (앞서 논의한 전략 패턴에 연결 가능)
    # gdp_table.to_csv("imf_gdp_latest.csv", index=False)

--- 2030년 기준 IMF GDP 데이터를 추출합니다 ---
     Country  GDP (1B USD)
0   WEOWORLD     149567.53
1      ADVEC      83750.24
2      OEMDC      65817.29
3        MAE      63052.36
4        APQ      54975.51
5        WHQ      48855.39
6        NMQ      42312.94
7         DA      38968.11
8        USA      36814.25
9        EUQ      36469.57
10       EAQ      35467.30
11       WEQ      29358.45
12       CHN      26336.99
13        EU      25781.92
14      EURO      21463.18
15       OAE      12236.59
16        WE       8984.77
17       SAQ       8095.93
18       EDE       7917.22
19       EEQ       7111.12


In [5]:
import requests
import pandas as pd

def get_imf_gdp_analysis():
    # 1. 추출 (Extract): IMF API에서 명목 GDP(NGDPD) 데이터 추출
    # API를 사용하면 매년 2회 갱신되는 데이터를 코드 수정 없이 항상 최신으로 유지 가능
    url = "https://www.imf.org/external/datamapper/api/v1/NGDPD"
    response = requests.get(url)
    data = response.json()

    gdp_values = data.get('values', {}).get('NGDPD', {})
    countries_meta = data.get('countries', {})

    # 2. 변환 (Transform): 최신 연도 파악 및 데이터 정제
    # 모든 국가 데이터 중 가장 최근 연도를 동적으로 찾음 (재사용성 보장)
    all_years = [year for country in gdp_values.values() for year in country.keys()]
    latest_year = max(all_years)

    processed_data = []
    for code, years in gdp_values.items():
        # ISO 3자리 국가 코드인 경우만 필터링 (지역 합계 및 그룹 데이터 제외)
        if len(code) == 3 and latest_year in years:
            country_info = countries_meta.get(code, {})
            processed_data.append({
                'Country': country_info.get('label', code),
                'Region': country_info.get('region', 'Other'),
                'GDP (1B USD)': round(float(years[latest_year]), 2)
            })

    # 데이터프레임 생성 및 정렬 (GDP 높은 순)
    df = pd.DataFrame(processed_data)
    df = df.sort_values(by='GDP (1B USD)', ascending=False).reset_index(drop=True)

    # --- 화면 출력용 분석 ---

    print(f"[{latest_year}년 기준 IMF GDP 분석 보고서]")
    print("-" * 50)

    # 결과 1: GDP가 100B USD 이상인 국가들
    print(f"\n1. GDP 100B USD 이상 국가 (총 {len(df[df['GDP (1B USD)'] >= 100])}개국):")
    over_100b = df[df['GDP (1B USD)'] >= 100].copy()
    over_100b.index = range(1, len(over_100b) + 1) # 순위 표시
    print(over_100b[['Country', 'GDP (1B USD)']].head(20)) # 상위 20개만 예시 출력

    # 결과 2: 각 Region별 Top 5 국가의 GDP 평균
    # 지역 코드 매핑 (보기 쉽게 변환)
    region_names = {
        'APD': 'Asia & Pacific', 'EUR': 'Europe', 'WHD': 'Western Hemisphere',
        'AFR': 'Africa', 'MCD': 'Middle East & Central Asia'
    }
    df['Region Name'] = df['Region'].map(region_names).fillna(df['Region'])

    print(f"\n2. 지역별 상위 5개국 GDP 평균 (단위: 1B USD):")
    region_avg = (
        df.groupby('Region Name')
        .apply(lambda x: x.nlargest(5, 'GDP (1B USD)')['GDP (1B USD)'].mean())
        .sort_values(ascending=False)
        .round(2)
    )
    print(region_avg)

# 데이터 확인 실행
get_imf_gdp_analysis()

[2030년 기준 IMF GDP 분석 보고서]
--------------------------------------------------

1. GDP 100B USD 이상 국가 (총 109개국):
   Country  GDP (1B USD)
1      MAE      63052.36
2      APQ      54975.51
3      WHQ      48855.39
4      NMQ      42312.94
5      USA      36814.25
6      EUQ      36469.57
7      EAQ      35467.30
8      WEQ      29358.45
9      CHN      26336.99
10     OAE      12236.59
11     SAQ       8095.93
12     EDE       7917.22
13     EEQ       7111.12
14     IND       6627.98
15     DEU       6013.75
16     SEQ       5712.59
17     SMQ       5607.25
18     GBR       5199.78
19     JPN       5119.89
20     MEQ       4856.09

2. 지역별 상위 5개국 GDP 평균 (단위: 1B USD):
Region Name
Other    49202.09
dtype: float64


/var/folders/86/7bdrddv528qd88kv5qsmsvlw0000gn/T/ipykernel_92561/2159209263.py:55: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('Region Name')


In [6]:
import requests
import pandas as pd

def get_imf_gdp_final_report():
    # 1. 데이터 추출
    url = "https://www.imf.org/external/datamapper/api/v1/NGDPD"
    response = requests.get(url)
    data = response.json()

    gdp_values = data.get('values', {}).get('NGDPD', {})
    countries_meta = data.get('countries', {})

    # 2. 최신 연도 탐색 (예: 2025년 또는 2030년 전망치)
    all_years = [year for country in gdp_values.values() for year in country.keys()]
    latest_year = max(all_years)

    # 3. 데이터 정제 (Transformation)
    # 지역 코드 매핑 (IMF 표준 지역명)
    region_map = {
        'AFR': 'Africa',
        'APD': 'Asia and Pacific',
        'EUR': 'Europe',
        'MCD': 'Middle East and Central Asia',
        'WHD': 'Western Hemisphere (Americas)'
    }

    clean_list = []
    for code, years in gdp_values.items():
        # [핵심] 3글자 ISO 국가 코드이고, 메타데이터에 실제 존재하는 국가만 필터링
        # (MAE, APQ, WEOWORLD 등은 3글자가 아니거나 지역 합계용 코드임)
        if len(code) == 3 and code in countries_meta and latest_year in years:
            meta = countries_meta[code]
            full_name = meta.get('label', code) # 코드 대신 이름 가져오기
            region_code = meta.get('region', 'Other')

            clean_list.append({
                'Country': full_name,
                'Region': region_map.get(region_code, region_code),
                'GDP (1B USD)': round(float(years[latest_year]), 2)
            })

    # 데이터프레임 생성 및 정렬
    df = pd.DataFrame(clean_list)
    df = df.sort_values(by='GDP (1B USD)', ascending=False).reset_index(drop=True)

    # --- 화면 출력부 ---
    print(f"\n[{latest_year}년 기준 IMF GDP 분석 보고서]")
    print("=" * 60)

    # [요구사항 1] GDP 100B USD 이상 국가만 출력
    df_over_100 = df[df['GDP (1B USD)'] >= 100].copy()
    df_over_100.index = range(1, len(df_over_100) + 1)
    print(f"\n1. GDP 100B USD 이상 국가 (총 {len(df_over_100)}개국)")
    print(df_over_100[['Country', 'GDP (1B USD)']].head(20)) # 상위 20개 출력

    # [요구사항 2] 지역별 Top 5 국가 GDP 평균
    # FutureWarning 해결: numeric_only 옵션 또는 컬럼 명시
    print(f"\n2. 지역별 상위 5개국 GDP 평균 (단위: 1B USD)")
    top5_avg_by_region = (
        df.groupby('Region')['GDP (1B USD)']
        .apply(lambda x: x.nlargest(5).mean())
        .sort_values(ascending=False)
        .round(2)
    )
    print(top5_avg_by_region)

# 실행
get_imf_gdp_final_report()

KeyError: 'GDP (1B USD)'

In [7]:
import requests
import pandas as pd

def get_gdp_analysis_report():
    # 1. Extraction: IMF DataMapper API 호출
    # NGDPD: GDP, Current Prices, US Dollars
    url = "https://www.imf.org/external/datamapper/api/v1/NGDPD"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        gdp_values = data.get('values', {}).get('NGDPD', {})
        countries_meta = data.get('countries', {})

        # 최신 연도 자동 탐색
        all_years = [y for c in gdp_values.values() for y in c.keys()]
        if not all_years:
            print("데이터를 찾을 수 없습니다.")
            return
        latest_year = max(all_years)

        # 2. Transformation: 데이터 정제 및 필터링
        # IMF 지역 코드 매핑
        region_map = {
            'AFR': 'Africa',
            'APD': 'Asia and Pacific',
            'EUR': 'Europe',
            'MCD': 'Middle East and Central Asia',
            'WHD': 'Western Hemisphere'
        }

        clean_list = []
        for code, years in gdp_values.items():
            # [필터링 핵심]
            # 1. ISO 3자리 코드이고 (WEOWORLD 등 제거)
            # 2. 메타데이터에 등록된 항목이며
            # 3. 그룹 데이터가 아닌 '국가'만 포함 (MAE, APQ 등 지역 합계 제외)
            if len(code) == 3 and code in countries_meta and latest_year in years:
                meta = countries_meta[code]
                full_name = meta.get('label', code)

                # 'economies', 'World', 'Area' 등이 포함된 레이블은 합계 데이터이므로 제외
                is_aggregate = any(word in full_name for word in ['economies', 'World', 'Area', 'Union', 'countries'])

                if not is_aggregate:
                    clean_list.append({
                        'Country': full_name,
                        'Region': region_map.get(meta.get('region'), 'Other'),
                        'GDP (1B USD)': round(float(years[latest_year]), 2)
                    })

        # 데이터프레임 생성
        df = pd.DataFrame(clean_list)

        # [오류 방지] 데이터가 없는 경우 처리
        if df.empty:
            print("조건에 맞는 국가 데이터가 없습니다.")
            return

        # GDP 내림차순 정렬
        df = df.sort_values(by='GDP (1B USD)', ascending=False).reset_index(drop=True)

        # 3. 화면 출력 (경영진 요구사항)
        print(f"\n[{latest_year}년 기준 IMF GDP 분석 보고서]")
        print("=" * 65)

        # 요구사항 1: GDP 100B USD 이상 국가
        print(f"\n1. GDP 100B USD 이상 국가 (총 {len(df[df['GDP (1B USD)'] >= 100])}개국)")
        print("-" * 65)
        over_100b = df[df['GDP (1B USD)'] >= 100].copy()
        over_100b.index = range(1, len(over_100b) + 1)
        print(over_100b[['Country', 'GDP (1B USD)']].head(20)) # 상위 20개 예시

        # 요구사항 2: 지역별 Top 5 평균 (FutureWarning 해결)
        print(f"\n2. 지역별 상위 5개국 GDP 평균 (단위: 1B USD)")
        print("-" * 65)
        region_stats = (
            df.groupby('Region')['GDP (1B USD)']
            .apply(lambda x: x.nlargest(5).mean())
            .sort_values(ascending=False)
            .round(2)
        )
        print(region_stats)

    except Exception as e:
        print(f"오류 발생: {e}")

# 실행
get_gdp_analysis_report()

조건에 맞는 국가 데이터가 없습니다.


In [8]:
import requests
import pandas as pd

def get_imf_gdp_report():
    # 1. Extraction: IMF DataMapper API 호출
    url = "https://www.imf.org/external/datamapper/api/v1/NGDPD"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        gdp_values = data.get('values', {}).get('NGDPD', {})
        countries_meta = data.get('countries', {})

        # 최신 연도 자동 탐색 (2030년 전망치 등 포함)
        all_years = [y for c_vals in gdp_values.values() for y in c_vals.keys()]
        if not all_years:
            print("데이터를 찾을 수 없습니다.")
            return
        latest_year = max(all_years)

        # IMF 공식 지역 코드 매핑 (이 코드가 있는 항목만 '국가'로 분류)
        region_labels = {
            'AFR': 'Africa',
            'APD': 'Asia and Pacific',
            'EUR': 'Europe',
            'MCD': 'Middle East and Central Asia',
            'WHD': 'Western Hemisphere'
        }

        clean_list = []
        for code, years in gdp_values.items():
            # 국가 메타데이터가 존재하고, 최신 연도 데이터가 있는 경우
            if code in countries_meta and latest_year in years:
                meta = countries_meta[code]
                region_code = meta.get('region')

                # [핵심] IMF 지역 코드가 할당된 항목만 '국가'로 간주 (MAE, World 등 제외)
                if region_code in region_labels:
                    full_name = meta.get('label', code) # 풀네임 가져오기
                    gdp_val = round(float(years[latest_year]), 2)

                    clean_list.append({
                        'Country': full_name,
                        'Region': region_labels[region_code],
                        'GDP (1B USD)': gdp_val
                    })

        # 2. DataFrame 생성 및 검증
        df = pd.DataFrame(clean_list)

        if df.empty:
            print("조건에 맞는 국가 데이터를 찾지 못했습니다. API 응답 구조를 확인해 주세요.")
            return

        # GDP 기준 내림차순 정렬
        df = df.sort_values(by='GDP (1B USD)', ascending=False).reset_index(drop=True)

        # 3. 화면 출력 (경영진 요구사항)
        print(f"\n[{latest_year}년 기준 IMF GDP 분석 보고서]")
        print("=" * 65)

        # 요구사항 1: GDP 100B USD 이상 국가 (풀네임 출력)
        df_100b = df[df['GDP (1B USD)'] >= 100].copy()
        df_100b.index = range(1, len(df_100b) + 1)
        print(f"\n1. GDP 100B USD 이상 국가 (총 {len(df_100b)}개국)")
        print("-" * 65)
        print(df_100b[['Country', 'GDP (1B USD)']].head(20)) # 상위 20개 출력

        # 요구사항 2: 지역별 상위 5개국 GDP 평균
        print(f"\n2. 지역별 상위 5개국 GDP 평균 (단위: 1B USD)")
        print("-" * 65)
        region_avg = (
            df.groupby('Region')['GDP (1B USD)']
            .apply(lambda x: x.nlargest(5).mean())
            .sort_values(ascending=False)
            .round(2)
        )
        print(region_avg)

    except Exception as e:
        print(f"실행 중 오류 발생: {e}")

# 실행
get_imf_gdp_report()

조건에 맞는 국가 데이터를 찾지 못했습니다. API 응답 구조를 확인해 주세요.


In [10]:
import requests
import pandas as pd

def generate_imf_gdp_report():
    # 1. API 엔드포인트 설정
    BASE_URL = "https://www.imf.org/external/datamapper/api/v1"
    GDP_URL = f"{BASE_URL}/NGDPD"
    COUNTRIES_URL = f"{BASE_URL}/countries"
    REGIONS_URL = f"{BASE_URL}/regions"

    try:
        # [Extract] 데이터 추출
        gdp_data = requests.get(GDP_URL).json()
        countries_meta = requests.get(COUNTRIES_URL).json().get('countries', {})
        regions_meta = requests.get(REGIONS_URL).json().get('regions', {})

        gdp_values = gdp_data.get('values', {}).get('NGDPD', {})

        # 최신 연도 자동 탐색 (2030년 등 전망치 포함)
        all_years = [y for c_vals in gdp_values.values() for y in c_vals.keys()]
        if not all_years:
            print("데이터를 찾을 수 없습니다.")
            return
        latest_year = max(all_years)

        # [Transform] 데이터 정제
        clean_list = []

        # 190여 개 개별 국가 정보를 기준으로 루프 실행
        for iso_code, meta in countries_meta.items():
            # 해당 국가의 GDP 수치가 최신 연도에 존재하는 경우
            if iso_code in gdp_values and latest_year in gdp_values[iso_code]:
                country_name = meta.get('label')
                region_id = meta.get('region') # 예: "1", "2"

                # 지역 ID를 실제 이름으로 매핑 (예: "1" -> "Africa")
                region_name = regions_meta.get(region_id, {}).get('label', 'Other')

                gdp_val = float(gdp_values[iso_code][latest_year])

                clean_list.append({
                    'Country': country_name,
                    'Region': region_name,
                    'GDP (1B USD)': round(gdp_val, 2)
                })

        # 데이터프레임 생성
        df = pd.DataFrame(clean_list)

        # 데이터가 비어있는지 확인 (KeyError 방지)
        if df.empty:
            print(f"{latest_year}년도에 해당하는 국가 데이터가 없습니다.")
            return

        # GDP 기준 내림차순 정렬
        df = df.sort_values(by='GDP (1B USD)', ascending=False).reset_index(drop=True)

        # [Load / Display] 화면 출력
        print(f"\n[{latest_year}년 기준 IMF GDP 분석 보고서]")
        print("=" * 70)

        # 1. GDP 100B USD 이상 국가 (높은 순 정렬)
        print(f"\n1. 시장 규모 100B USD 이상 국가 (총 {len(df[df['GDP (1B USD)'] >= 100])}개국)")
        print("-" * 70)
        report_1 = df[df['GDP (1B USD)'] >= 100].copy()
        report_1.index = range(1, len(report_1) + 1)
        # 전체를 보고 싶으시면 head()를 제거하거나 숫자를 늘리세요
        print(report_1[['Country', 'GDP (1B USD)']].head(30))

        # 2. 각 Region별 top5 국가의 GDP 평균
        print(f"\n2. 지역별 상위 5개국(Top 5) GDP 평균액 (단위: 1B USD)")
        print("-" * 70)

        # 지역별로 그룹화하여 상위 5개 값만 추출 후 평균 계산
        region_analysis = (
            df.groupby('Region')['GDP (1B USD)']
            .apply(lambda x: x.nlargest(5).mean())
            .sort_values(ascending=False)
            .round(2)
        )
        print(region_analysis.to_string())

    except Exception as e:
        print(f"오류 발생: {e}")

if __name__ == "__main__":
    generate_imf_gdp_report()


[2030년 기준 IMF GDP 분석 보고서]

1. 시장 규모 100B USD 이상 국가 (총 86개국)
----------------------------------------------------------------------
                        Country  GDP (1B USD)
1                 United States      36814.25
2   China, People's Republic of      26336.99
3                         India       6627.98
4                       Germany       6013.75
5                United Kingdom       5199.78
6                         Japan       5119.89
7                        France       4009.17
8                         Italy       2985.05
9                        Canada       2910.47
10           Russian Federation       2829.86
11                       Brazil       2814.46
12                       Mexico       2442.32
13                        Spain       2380.02
14                    Australia       2335.26
15           Korea, Republic of       2271.15
16                    Indonesia       2079.14
17         Türkiye, Republic of       1961.78
18                  Netherlands       16

In [12]:
import requests
import pandas as pd

def get_imf_gdp_final_report():
    # 1. API 엔드포인트 설정
    BASE_URL = "https://www.imf.org/external/datamapper/api/v1"
    GDP_URL = f"{BASE_URL}/NGDPD"      # GDP 수치
    COUNTRIES_URL = f"{BASE_URL}/countries"  # 국가 상세 정보
    REGIONS_URL = f"{BASE_URL}/regions"      # 지역 명칭 정보

    try:
        # [Extract] 데이터 추출
        gdp_res = requests.get(GDP_URL).json()
        countries_res = requests.get(COUNTRIES_URL).json().get('countries', {})
        regions_res = requests.get(REGIONS_URL).json().get('regions', {})

        gdp_values = gdp_res.get('values', {}).get('NGDPD', {})

        # 최신 연도 자동 탐색 (2030년 등 전망치 포함)
        all_years = [y for c_vals in gdp_values.values() for y in c_vals.keys()]
        if not all_years: return
        latest_year = max(all_years)

        # [Transform] 데이터 정제 및 매핑
        clean_list = []

        # IMF의 5대 핵심 지역 코드 (이 영역에 속해야만 '개별 국가'로 간주)
        # 1:Africa, 2:Asia and Pacific, 3:Europe, 4:Middle East/Central Asia, 5:Western Hemisphere
        valid_region_ids = ["1", "2", "3", "4", "5"]

        for iso_code, years in gdp_values.items():
            # 1. 해당 국가의 최신 수치가 있고, 국가 메타데이터가 존재하는 경우
            if latest_year in years and iso_code in countries_res:
                meta = countries_res[iso_code]
                region_id = str(meta.get('region', '')) # ID를 문자열로 통일

                # 2. 지역 정보가 유효한 '국가'만 필터링 (MAE, G7 등 그룹 제거)
                if region_id in valid_region_ids:
                    country_name = meta.get('label', iso_code) # 국가 풀네임
                    region_name = regions_res.get(region_id, {}).get('label', 'Other')
                    gdp_val = round(float(years[latest_year]), 2)

                    clean_list.append({
                        'Country': country_name,
                        'Region': region_name,
                        'GDP (1B USD)': gdp_val
                    })

        # 데이터프레임 생성
        df = pd.DataFrame(clean_list)

        # [오류 방지] 데이터 확인 후 정렬
        if df.empty:
            print("조건에 맞는 데이터를 찾지 못했습니다.")
            return

        df = df.sort_values(by='GDP (1B USD)', ascending=False).reset_index(drop=True)

        # [Load / Display] 화면 출력
        print(f"\n[{latest_year}년 기준 IMF GDP 분석 보고서]")
        print("=" * 75)

        # 요구사항 1: GDP 100B USD 이상 국가 (풀네임 정렬)
        df_100b = df[df['GDP (1B USD)'] >= 100].copy()
        df_100b.index = range(1, len(df_100b) + 1)
        print(f"\n1. 시장 규모 100B USD 이상 국가 (총 {len(df_100b)}개국)")
        print("-" * 75)
        # 상위 20개국 출력 (미국, 중국 등 풀네임 확인 가능)
        print(df_100b[['Country', 'Region', 'GDP (1B USD)']].head(20).to_string())

        # 요구사항 2: 지역별 Top 5 국가 평균
        print(f"\n2. 지역별 상위 5개국(Top 5) GDP 평균액 (단위: 1B USD)")
        print("-" * 75)

        # FutureWarning 방지 및 정확한 그룹 연산
        region_avg = (
            df.groupby('Region')['GDP (1B USD)']
            .apply(lambda x: x.nlargest(5).mean())
            .sort_values(ascending=False)
            .round(2)
        )
        print(region_avg.to_string())

    except Exception as e:
        print(f"오류 발생: {e}")

if __name__ == "__main__":
    get_imf_gdp_final_report()

조건에 맞는 데이터를 찾지 못했습니다.


In [13]:
import requests
import pandas as pd

def run_imf_gdp_pipeline():
    # 1. API 엔드포인트 설정
    BASE_URL = "https://www.imf.org/external/datamapper/api/v1"
    GDP_URL = f"{BASE_URL}/NGDPD"      # 명목 GDP 수치
    COUNTRIES_URL = f"{BASE_URL}/countries"  # 국가 풀네임 및 지역 ID 정보
    REGIONS_URL = f"{BASE_URL}/regions"      # 지역 ID별 실제 명칭

    try:
        # [Extract] 데이터 추출
        print("IMF API로부터 최신 데이터를 추출 중입니다...")
        gdp_res = requests.get(GDP_URL).json()
        countries_res = requests.get(COUNTRIES_URL).json().get('countries', {})
        regions_res = requests.get(REGIONS_URL).json().get('regions', {})

        gdp_values = gdp_res.get('values', {}).get('NGDPD', {})

        # 최신 연도 자동 탐색 (2025년, 2030년 전망치 등 대응)
        all_years = [y for c_vals in gdp_values.values() for y in c_vals.keys()]
        if not all_years:
            print("데이터를 찾을 수 없습니다.")
            return
        latest_year = max(all_years)

        # [Transform] 데이터 병합 및 정제
        clean_list = []

        # 개별 국가(ISO 3자리)를 기준으로 루프 실행
        for iso_code, meta in countries_res.items():
            # 1. 3자리 국가 코드인 경우만 처리 (그룹 데이터 제외)
            # 2. 해당 국가의 최신 연도 GDP 데이터가 존재하는 경우
            if len(iso_code) == 3 and iso_code in gdp_values and latest_year in gdp_values[iso_code]:
                country_label = meta.get('label', iso_code)
                region_id = str(meta.get('region', '')) # ID를 문자열로 통일하여 매핑 정확도 향상

                # 지역 ID를 통해 지역명 가져오기 (매핑 실패 시 'Other' 혹은 'Unclassified')
                region_label = regions_res.get(region_id, {}).get('label', 'Other/Aggregates')

                # 그룹 데이터(World, Advanced Economies 등)가 이름에 포함된 경우 제외
                if any(word in region_label for word in ['World', 'Group', 'income']):
                    continue

                gdp_val = round(float(gdp_values[iso_code][latest_year]), 2)

                clean_list.append({
                    'Country': country_label,
                    'Region': region_label,
                    'GDP (1B USD)': gdp_val
                })

        # 데이터프레임 생성
        df = pd.DataFrame(clean_list)

        if df.empty:
            print("필터링 조건에 맞는 국가 데이터를 찾지 못했습니다. API 응답을 재확인하세요.")
            return

        # GDP 기준 내림차순 정렬
        df = df.sort_values(by='GDP (1B USD)', ascending=False).reset_index(drop=True)

        # [Load / Display] 화면 출력부
        print(f"\n[{latest_year}년 기준 IMF GDP 분석 보고서]")
        print("=" * 70)

        # 요구사항 1: GDP 100B USD 이상 국가 (풀네임 리스트)
        df_100b = df[df['GDP (1B USD)'] >= 100].copy()
        df_100b.index = range(1, len(df_100b) + 1)
        print(f"\n1. 시장 규모 100B USD 이상 국가 (총 {len(df_100b)}개국)")
        print("-" * 70)
        # 상위 20개 국가 확인
        print(df_100b[['Country', 'Region', 'GDP (1B USD)']].head(20).to_string())

        # 요구사항 2: 각 Region별 top5 국가의 GDP 평균
        print(f"\n2. 지역별 상위 5개국(Top 5) GDP 평균액 (단위: 1B USD)")
        print("-" * 70)

        region_avg = (
            df.groupby('Region')['GDP (1B USD)']
            .apply(lambda x: x.nlargest(5).mean())
            .sort_values(ascending=False)
            .round(2)
        )
        print(region_avg.to_string())

    except Exception as e:
        print(f"데이터 파이프라인 실행 중 오류 발생: {e}")

if __name__ == "__main__":
    run_imf_gdp_pipeline()

IMF API로부터 최신 데이터를 추출 중입니다...

[2030년 기준 IMF GDP 분석 보고서]

1. 시장 규모 100B USD 이상 국가 (총 86개국)
----------------------------------------------------------------------
                        Country            Region  GDP (1B USD)
1                 United States  Other/Aggregates      36814.25
2   China, People's Republic of  Other/Aggregates      26336.99
3                         India  Other/Aggregates       6627.98
4                       Germany  Other/Aggregates       6013.75
5                United Kingdom  Other/Aggregates       5199.78
6                         Japan  Other/Aggregates       5119.89
7                        France  Other/Aggregates       4009.17
8                         Italy  Other/Aggregates       2985.05
9                        Canada  Other/Aggregates       2910.47
10           Russian Federation  Other/Aggregates       2829.86
11                       Brazil  Other/Aggregates       2814.46
12                       Mexico  Other/Aggregates       2442.32
13    